<h1 style="background-color:#FF7733;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
    🌐 תהליך הסבת חלקות (דו-ממדיות) בתהליך לבנק"ל מודרני 🌐

In [ ]:
print(' In-Process Parcels Borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [1]:
import warnings
warnings.filterwarnings('ignore')

import subprocess
import numpy as np
import pandas as pd
import geopandas as gpd
from configs import CNFG
from threading import Thread
import dask_geopandas as daskgpd
from helpers import assign_ProcessName, assign_BlockName

<h1 style="text-align:right;font-size:100%">
העתקת שכבת מקור שכוללת גיאומטריה מקורית

In [ ]:
print('   Preprocess')

In [ ]:
Thread(target = subprocess.call, args = [CNFG.BatchPath + "Parcels2D_InProcess_preprocess_calc.bat"]).start();

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת חלקות בתהליך קיימת

In [2]:
Parcels_InProcess_cols = ['TEMP_ID', 'PARCELNAME', 'GUSHNUM', 'GUSHSUFFIX', 'TALAR_NUM', 'TALAR_YEAR', 'CADASTER_PROCESS', 'ENTITY_STATE', 'TABAYEUD', 'LEGALAREA']

Parcels_InProcess = pd.read_csv(CNFG.CADSDE + 'Parcels_InProcess.csv', usecols = Parcels_InProcess_cols)

del Parcels_InProcess_cols
print('       Input 1/5 read')

       Input 1/5 read


<h1 style="text-align:right;font-size:100%">
טבלת פעולות של תכניות בתהליך (לחילוץ חלקות ביניים)

In [ ]:
MiddleParcels_cols = ['PARCEL_ID', 'TALAR_NUM', 'TALAR_YEAR', 'GUSHNUM', 'GUSHSUFFIX', 'ACTION_TYPE', 'PARCEL_AREA']

MiddleParcels = pd.read_csv(CNFG.Oracle + 'table_inprocess.csv', usecols = MiddleParcels_cols)

del MiddleParcels_cols
print('       Input 2/5 read')

<h1 style="text-align:right;font-size:100%">
שכבת גושי קדסטר מודרנית

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'GlobalID', 'IsTax'])
print('       Input 3/5 read')

<h1 style="text-align:right;font-size:100%">
שמות תת"גים כפולים

In [ ]:
duplicated_names = pd.read_excel(CNFG.Local + 'tatag_duplicated_names.xlsx')['TatagName']
print('       Input 4/5 read')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות תהליכי קדסטר מודרנית

In [ ]:
cpb_cols = ['ProcessGUID', 'ProcessName', 'ProcessType']

cpb_layer = pd.read_csv(CNFG.CSVPath + 'ProcessesAndRecords.csv', usecols = cpb_cols)

del cpb_cols
print('       Input 5/5 read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('   Cleaning data')

<h1 style="text-align:right;font-size:100%">
פיצול טבלת גושים לגושי קדסטר וגושי שומא

In [ ]:
cadaster_Blocks = Blocks[Blocks['IsTax'] == 0].drop(columns = 'IsTax').rename(columns = {'GlobalID' : 'cadasterBlockUniqueID', 'Name' : 'BlockName'})
tax_Blocks      = Blocks[Blocks['IsTax'] == 1].rename(columns = {'GlobalID' : 'taxBlockUniqueID', 'Name' : 'BlockName'})

In [ ]:
print('   Data conversion')

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני חלקות בתהליך

<h1 style="text-align:right;font-size:100%">
.מיזוג עם גושי קדסטר ולאחר מכן עם גושי שומא לקבלת מזהה גוש ושדה של תיאור שומא. במקרים בהם המיזוג תואם לשני סוגי גושים, גוש הקדסטר יקבע

In [ ]:
Parcels_InProcess = assign_BlockName(Parcels_InProcess, 'GUSHNUM', 'GUSHSUFFIX', drop=False).rename(columns={'GUSHNUM':'BlockNumber', 'GUSHSUFFIX':'SubBlockNumber'})

Parcels_InProcess = Parcels_InProcess.merge(cadaster_Blocks, on = 'BlockName', how = 'left')\
                                     .merge(tax_Blocks, on = 'BlockName', how = 'left')

Parcels_InProcess['BlockUniqueID'] = Parcels_InProcess['cadasterBlockUniqueID'].fillna(Parcels_InProcess['taxBlockUniqueID'])

Parcels_InProcess['IsTax'] = Parcels_InProcess['IsTax'].fillna(0)
Parcels_InProcess.drop(columns = ['cadasterBlockUniqueID', 'taxBlockUniqueID'], inplace = True)

<h1 style="text-align:right;font-size:100%">
מילוי ערכי סוג תהליך ב-1 (הסדר מקרקעין)

In [ ]:
Parcels_InProcess['CADASTER_PROCESS'] = Parcels_InProcess['CADASTER_PROCESS'].fillna(1) #הסדר מקרקעין
Parcels_InProcess['CADASTER_PROCESS'] = Parcels_InProcess['CADASTER_PROCESS'].astype(int)

<h1 style="text-align:right;font-size:100%">
הגדרת שם מפה לצורך קבלת מזהה תהליך קדסטרי
<h1 style="text-align:right;font-size:100%">
חלקות ללא שם מפה אשר מגיעות מתהליך מסוג 1 (הסדר) יקבלו שם מפת הסדר לפי מספר גוש מלא
<h1 style="text-align:right;font-size:100%">
חלקות ללא שם מפה אשר מגיעות מתהליך מסוג 1 (קמ"ק) יקבלו שם מפת קמ"ק לפי מספר גוש מלא
<h1 style="text-align:right;font-size:100%">
תיקון שמות מפה כפולים של תת"ג עם הוספת קידומת 0 לשם המפה
<h1 style="text-align:right;font-size:100%">
מיזוג נתונים עם טבלת גבולות תכניות קדסטריות לקבלת מזהה תהליך וסוג תהליך עדכני, הסרת שדות שאינם רלוונטיים יותר

In [ ]:
Parcels_InProcess[['TALAR_NUM', 'TALAR_YEAR']] = Parcels_InProcess[['TALAR_NUM', 'TALAR_YEAR']].fillna(0)
Parcels_InProcess[['TALAR_NUM', 'TALAR_YEAR']] = Parcels_InProcess[['TALAR_NUM', 'TALAR_YEAR']].astype(int)
Parcels_InProcess['ProcessName'] = Parcels_InProcess['TALAR_NUM'].astype(str) + '/' + Parcels_InProcess['TALAR_YEAR'].astype(str)
Parcels_InProcess['ProcessName'] = Parcels_InProcess['ProcessName'].replace('0/0', None)

#הסדר מקרקעין
Parcels_InProcess['ProcessName'] = np.where(Parcels_InProcess['CADASTER_PROCESS'] == 1, #הסדר מקרקעין
                                            Parcels_InProcess['BlockName'] + ' הסדר',
                                            Parcels_InProcess['ProcessName'])

#קמ"ק
Parcels_InProcess['ProcessName'] = np.where(Parcels_InProcess['CADASTER_PROCESS'] == 9, #טיוב גוש אנליטי (קמ"ק)
                                            'קמ"ק ' + Parcels_InProcess['BlockName'],
                                            Parcels_InProcess['ProcessName'])


duplicated_cond = (Parcels_InProcess['ProcessName'].isin(duplicated_names)) & (Parcels_InProcess['CADASTER_PROCESS'].isin([3,12])) #תת"ג

Parcels_InProcess['ProcessName'] = np.where(duplicated_cond, '0' + Parcels_InProcess['ProcessName'], Parcels_InProcess['ProcessName'])


Parcels_InProcess = Parcels_InProcess.merge(right = cpb_layer, on = 'ProcessName', how = 'left')\
                                     .rename(columns = {'ProcessGUID' : 'CPBUniqueID'})\
                                     .drop(columns = ['ProcessName', 'CADASTER_PROCESS', 'TALAR_NUM', 'TALAR_YEAR'])


del [duplicated_names, duplicated_cond]

<h1 style="text-align:right;font-size:100%">
:הגדרת שדה של מעמד החלקה
<h1 style="text-align:right;font-size:100%">
הסרת חלקות שמשמשות כחלקות רקע 
<h1 style="text-align:right;font-size:100%">
התאמת ערכי מעמד החלקה לערכים חדשים על ידי הוספת 1
<h1 style="text-align:right;font-size:100%">
חלקות שמגיעות מתהליך מעדכן יקבלו מעמד חלקה 3 (ביסוס וחדשה)
<h1 style="text-align:right;font-size:100%">
חלקות שמגיעות מתהליך יוצר יקבלו מעמד חלקה 2 (חדשה)

In [ ]:
Parcels_InProcess = Parcels_InProcess[Parcels_InProcess['ENTITY_STATE'].isin([0, 1])]
Parcels_InProcess.dropna(subset = ['ENTITY_STATE'], inplace = True)
Parcels_InProcess['ENTITY_STATE'] = Parcels_InProcess['ENTITY_STATE'] + 1

UpdatingProcesses_cond = Parcels_InProcess['ProcessType'].isin([4, 5])
CreatingProcesses_cond = Parcels_InProcess['ProcessType'] == 9

Parcels_InProcess['ENTITY_STATE'] = np.where(UpdatingProcesses_cond, 3, Parcels_InProcess['ENTITY_STATE'])
Parcels_InProcess['ENTITY_STATE'] = np.where(CreatingProcesses_cond, 2, Parcels_InProcess['ENTITY_STATE'])


Parcels_InProcess['ENTITY_STATE'] = Parcels_InProcess['ENTITY_STATE'].astype(int)
Parcels_InProcess.rename(columns = {'ENTITY_STATE' : 'ParcelRole'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של סוג חלקה:
<h1 style="text-align:right;font-size:100%">
שמירת חלקות בתהליך שמכילות מידע רלוונטי לארעיות או סופיות בלבד, ערכים ריקים יוגדרו כחלקות בתהליך סופיות 
<h1 style="text-align:right;font-size:100%">
החלפת ערכי אותיות למספרים התואמים

In [ ]:
Parcels_InProcess['ParcelType'] = Parcels_InProcess['PARCELNAME'].str.get(0)
Parcels_InProcess['ParcelType'] = Parcels_InProcess['ParcelType'].astype(str)


unknown_condition = ~Parcels_InProcess['ParcelType'].isin(['T', 'F'])
Parcels_InProcess['ParcelType'] = np.where(unknown_condition, 'F', Parcels_InProcess['ParcelType'])


Parcels_InProcess['ParcelType'] = Parcels_InProcess['ParcelType'].map({'T' : 1, 'F' : 2})

del unknown_condition

<h1 style="text-align:right;font-size:100%">
חילוץ מספר החלקה ללא אותיות

In [ ]:
Parcels_InProcess['ParcelNumber'] = Parcels_InProcess['PARCELNAME'].map(lambda x: x.lstrip('FT'))
Parcels_InProcess['ParcelNumber'] = Parcels_InProcess['ParcelNumber'].str.partition(":")[0]
Parcels_InProcess['ParcelNumber'] = Parcels_InProcess['ParcelNumber'].astype(int)
Parcels_InProcess.drop(columns = 'PARCELNAME', inplace = True)

<h1 style="text-align:right;font-size:100%">
הכפלת שטח רשום פי 1000
<h1 style="text-align:right;font-size:100%">
שינוי שם שדה של שטח רשום וייעוד קרקע

In [ ]:
Parcels_InProcess['LEGALAREA'] = Parcels_InProcess['LEGALAREA'] * 1000
Parcels_InProcess.rename(columns = {'LEGALAREA' : 'LegalArea', 'TABAYEUD' : 'LandDesignationPlan'}, inplace = True)

<h1 style="text-align:right;font-size:100%">
הגדרת שדה של סוג מקרקעין לפי סוג תהליך קדסטרי

In [ ]:
processes_in_registered = Parcels_InProcess['ProcessType'].isin([1, 4, 5])

Parcels_InProcess['LandType'] = np.where(processes_in_registered, 1, 2)

<h1 style="text-align:right;font-size:125%;">
:עיבוד נתוני חלקות ביניים בתהליך

<h1 style="text-align:right;font-size:100%;">
חילוץ מספרים מלאים של חלקות בתהליך לצורך סינון בהמשך

In [ ]:
Parcels_InProcess['ParcelNumber'] = Parcels_InProcess['ParcelNumber'].astype(int)
Parcels_InProcess['ParcelName'] = Parcels_InProcess['ParcelNumber'].astype(str) + Parcels_InProcess['BlockName']
not_middle = list(Parcels_InProcess['ParcelNumber'].astype(str) + '/' + Parcels_InProcess['BlockName'])

Parcels_InProcess.drop(columns = ['BlockName', 'ParcelName'], inplace = True)

<h1 style="text-align:right;font-size:100%;">
סינון חלקות שמתארות פעולת העברה
<h1 style="text-align:right;font-size:100%;">
חילוץ מספר חלקה
<h1 style="text-align:right;font-size:100%;">
הגדרת שדה של שם מפה
<h1 style="text-align:right;font-size:100%;">
הגדרת שדה של מספר גוש מלא
<h1 style="text-align:right;font-size:100%;">
הגדרת שדה של שם חלקה מלא בחלקות הביניים
<h1 style="text-align:right;font-size:100%;">
סינון חלקות ביניים שהן אינן מופיעות בחלקות בתהליך
<h1 style="text-align:right;font-size:100%;">
שינוי שם שדה של שטח רשום
<h1 style="text-align:right;font-size:100%;">
הגדרת שדה של סוג חלקה 
<h1 style="text-align:right;font-size:100%;">
הגדרת שדה של מעמד חלקה (ביניים)
<h1 style="text-align:right;font-size:100%;">
הסרת שדות שאינם רלונטיים יותר

In [ ]:
MiddleParcels = MiddleParcels[MiddleParcels['ACTION_TYPE'] != 'transfer']

MiddleParcels['ParcelNumber'] = MiddleParcels['PARCEL_ID'].str.replace(r'\D', '', regex = True)
MiddleParcels['ParcelNumber'] = MiddleParcels['ParcelNumber'].astype(int)
MiddleParcels['ParcelMode'] = MiddleParcels['PARCEL_ID'].str.get(0)

MiddleParcels = assign_ProcessName(MiddleParcels, 'TALAR_NUM', 'TALAR_YEAR')
MiddleParcels = assign_BlockName(MiddleParcels, 'GUSHNUM', 'GUSHSUFFIX', drop=False).rename(columns={'GUSHNUM':'BlockNumber', 'GUSHSUFFIX':'SubBlockNumber'})
MiddleParcels['ParcelName'] = MiddleParcels['ParcelNumber'].astype(str) + '/' + MiddleParcels['BlockName']
MiddleParcels = MiddleParcels[MiddleParcels['ParcelName'].notna()]
MiddleParcels = MiddleParcels[~MiddleParcels['ParcelName'].isin(not_middle)]
del not_middle

MiddleParcels.rename(columns = {'PARCEL_AREA':'LegalArea'}, inplace = True)

MiddleParcels['ParcelType'] = np.where(MiddleParcels['ParcelMode'] == 'T', 1, 2) #T == ארעית, אחרת סופית
MiddleParcels['ParcelRole'] = 4 #ביניים

MiddleParcels.drop(columns = ['PARCEL_ID','ACTION_TYPE', 'ParcelMode', 'ParcelName'], inplace = True)

<h1 style="text-align:right;font-size:100%;">
מיזוג חלקות ביניים עם תהליכי קדסטר לקבלת מזהה תהליך וסוג תהליך
<h1 style="text-align:right;font-size:100%;">
מיזוג עם גושי קדסטר ולאחר מכן עם גושי שומא
<h1 style="text-align:right;font-size:100%;">
הגדרת שדה סוג מקרקעין לפי סוג התהליך
<h1 style="text-align:right;font-size:100%;">
הסרת שדות שאין בהם יותר צורך

In [ ]:
MiddleParcels = MiddleParcels.merge(cpb_layer, on = 'ProcessName', how = 'left')\
                             .rename(columns = {'ProcessGUID' : 'CPBUniqueID'})

MiddleParcels = MiddleParcels.merge(cadaster_Blocks, on = 'BlockName', how = 'left')\
                             .merge(tax_Blocks, on = 'BlockName', how = 'left')

MiddleParcels['BlockUniqueID'] = MiddleParcels['cadasterBlockUniqueID'].fillna(MiddleParcels['taxBlockUniqueID'])

MiddleParcels['IsTax'] = MiddleParcels['IsTax'].fillna(0)

MiddleParcels['LandType'] = np.where(MiddleParcels['ProcessType']==1, 1, 2)


MiddleParcels.drop(columns = ['cadasterBlockUniqueID', 'taxBlockUniqueID', 'BlockName', 'ProcessName'], inplace = True)

del [Blocks, cpb_layer, cadaster_Blocks, tax_Blocks]

<h1 style="text-align:right;font-size:125%;">
הגדרות שדות של מספרים שלמלים כמספרים שלמים לשתי הטבלאות

In [ ]:
int1_cols = ['ProcessType', 'ParcelRole', 'TEMP_ID', 'ParcelType', 'ParcelNumber', 'LandType', 'IsTax', 'BlockNumber', 'SubBlockNumber']
int2_cols = ['ProcessType', 'ParcelRole', 'ParcelType', 'ParcelNumber', 'LandType', 'IsTax', 'BlockNumber', 'SubBlockNumber']

Parcels_InProcess[int1_cols] = Parcels_InProcess[int1_cols].astype('Int64')
MiddleParcels[int2_cols] = MiddleParcels[int2_cols].astype('Int64')

del [int1_cols, int2_cols]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾

In [ ]:
print('   Exporting results')

In [ ]:
Parcels_InProcess.to_excel(CNFG.CSVPath + 'InProcessParcels2D_data_join.xlsx', index = False)

MiddleParcels.to_excel(CNFG.CSVPath + 'InProcessParcels2D_data_append.xlsx', index = False)

In [ ]:
subprocess.call([CNFG.BatchPath + "Parcels2D In-Process-Save as Feature Class.bat"]);